<a href="https://colab.research.google.com/github/shivathmika339/vader-lexicon-analysis/blob/main/Using_Vader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install contractions
!pip install vaderSentiment

In [ ]:
import requests
import pandas as pd
import numpy as np
import os
import contractions
import re

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
from nltk.tokenize.toktok import ToktokTokenizer

tokenizer = ToktokTokenizer()
analyzer = SentimentIntensityAnalyzer()

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [ ]:
import nltk
nltk.download('stopwords')
stopwordslist = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stopwordslist.remove('no')
stopwordslist.remove('not')

In [ ]:
def Preprocess_dataset(text):
  text = text.lower()

  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filteredtokens = [token for token in tokens if token not in stopwordslist]
  filteredtokens = ' '.join(filteredtokens)

  expand = contractions.fix(filteredtokens)

  pattern = r'[^a-zA-Z0-9\s]'
  new_text = re.sub(pattern,'',expand)

  return new_text

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Major Project/News Articles.csv')

In [ ]:
dataset.news_article = dataset.news_article.apply(Preprocess_dataset)

In [ ]:
dataset.news_headline = dataset.news_headline.apply(Preprocess_dataset)

In [ ]:
dataset['Postive'] = dataset['news_article'].apply(lambda x: analyzer.polarity_scores(x)['pos'])
dataset['Neutral'] = dataset['news_article'].apply(lambda x: analyzer.polarity_scores(x)['neu'])
dataset['Negative'] = dataset['news_article'].apply(lambda x: analyzer.polarity_scores(x)['neg'])

In [ ]:
dataset['Compound'] = dataset['news_article'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
dataset

,news_headline,news_article,news_category,Postive,Neutral,Negative,Compound
0,"Wrestler Sushil asks for protein supplements, ...",olympic medalwinning wrestler sushil kumar ar...,sports,0.095,0.714,0.190,-0.7003
1,"Manjrekar's chat with Twitter user goes viral,...",twitter user shared screenshots personal chat ...,sports,0.062,0.938,0.000,0.3400
2,"Sometimes, we didn't want to see each other: P...",exbcci chief selector msk prasad revealed sele...,sports,0.222,0.702,0.076,0.6369
3,I don't count goals: Chhetri on surpassing Mes...,asked counting number international goals ind...,sports,0.114,0.886,0.000,0.4588
4,"After French Open, world no. 2 Osaka withdraws...",women world number two naomi osaka monday wit...,sports,0.168,0.779,0.053,0.6808
...,...,...,...,...,...,...,...
70,I printed out negative feedback when Reddit wa...,reddit cofounder alexis ohanian took twitter s...,technology,0.119,0.583,0.298,-0.8908
71,Indian investors made $241 mn profit from Bitc...,us topped list countries earned investing bitc...,technology,0.209,0.791,0.000,0.8271
72,Hong Kong to test linking China's digital yuan...,hong kong test linking domestic payments netwo...,technology,0.000,1.000,0.000,0.0000
73,Dell sued over 'misleading' claims about Alien...,dell facing lawsuit us running false misleadi...,technology,0.115,0.736,0.149,-0.1280


In [ ]:
def func(n):
  if n >= 0.05:
    return 'Positive'
  elif n > -0.05 and n < 0.05:
    return 'Neutral'
  elif n <= -0.05:
    return 'Negative'
dataset.Compound = dataset.Compound.apply(func)

In [ ]:
dataset

,news_headline,news_article,news_category,Postive,Neutral,Negative,Compound
0,wrestler sushil asks protein supplements exer...,olympic medalwinning wrestler sushil kumar ar...,sports,0.095,0.714,0.190,Negative
1,manjrekar chat twitter user goes viral says ...,twitter user shared screenshots personal chat ...,sports,0.062,0.938,0.000,Positive
2,sometimes want see prasad arguments kohli ...,exbcci chief selector msk prasad revealed sele...,sports,0.222,0.702,0.076,Positive
3,count goals chhetri surpassing messi intern...,asked counting number international goals ind...,sports,0.114,0.886,0.000,Positive
4,french open world no 2 osaka withdraws prewim...,women world number two naomi osaka monday wit...,sports,0.168,0.779,0.053,Positive
...,...,...,...,...,...,...,...
70,printed negative feedback reddit launched cof...,reddit cofounder alexis ohanian took twitter s...,technology,0.119,0.583,0.298,Negative
71,indian investors made 241 mn profit bitcoin 2...,us topped list countries earned investing bitc...,technology,0.209,0.791,0.000,Positive
72,hong kong test linking china digital yuan dom...,hong kong test linking domestic payments netwo...,technology,0.000,1.000,0.000,Neutral
73,dell sued misleading claims alienware laptop us,dell facing lawsuit us running false misleadi...,technology,0.115,0.736,0.149,Negative


In [ ]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1
from pyngrok import ngrok

     |████████████████████████████████| 7.8MB 7.6MB/s 
     |████████████████████████████████| 4.2MB 49.5MB/s 
     |████████████████████████████████| 174kB 55.5MB/s 
     |████████████████████████████████| 112kB 56.0MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
     |████████████████████████████████| 122kB 49.2MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompatible.
  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15985 sha256=3f8d73fa96ae31036b948f14811842ed62eae111c2e01d099b7dea5002478ae3
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

st.title("Sentimental Analysis Using Lexicon Based Approach...")
st.header("Please use proper spelling!")

iput = st.text_input("Enter Text:")
oput_dict =  analyzer.polarity_scores(iput)
col1, col2, col3 = st.beta_columns(3)
if st.button('Analyze'):
  with col1:
    st.header('**Negative** :unamused:')
    st.header(oput_dict['neg']*100)
  with col2:
    st.header('**Neutral** :unamused:')
    st.header(oput_dict['neu']*100)
  with col3:
    st.header('**Positive** :smile:')
    st.header(oput_dict['pos']*100)
  if oput_dict['compound'] >= 0.05:
    st.write("Its ",int(oput_dict['pos']*100),"% Positive.")
  elif oput_dict['compound'] <= -0.05:
    st.write("Its ",int(oput_dict['neg']*100),"% Negative.")
  else:
    st.write("Its ",int(oput_dict['neu']*100),"% Neutral.")
   


Overwriting app.py


In [ ]:
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores('hey')

#{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

# st.header('**_Neutral_**',oput_dict['neu'],':unamused:')
   # st.write('Neutral : ',oput_dict['neu']*100,'% :unamused:')
   # st.write('Positive : ',oput_dict['pos'],'% :smile:')
   # st.write('Negative',oput_dict['neg'],'% :angry:')


{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [ ]:
!nohup streamlit run app.py &
url = ngrok.connect(port = '8501')


nohup: appending output to 'nohup.out'


t=2021-06-22T10:25:06+0000 lvl=warn msg="failed to start tunnel" pg=/api/tunnels id=68a4c8972b50c2e8 err="Your account may not run more than 2 tunnels over a single ngrok client session.\nThe tunnels already running on this session are:\ntn_1uIPboMDnz4WRQDR8nTUipoGkRw, tn_1uIPbgbZgZyBSEtqr0uwTwXlRaK\n\r\n\r\nERR_NGROK_324\r\n"



PyngrokNgrokHTTPError: ignored

In [ ]:
url

'http://8897944b6451.ngrok.io'

In [ ]:
import sys
sys.version

'3.7.10 (default, May  3 2021, 02:48:31) \n[GCC 7.5.0]'